## The Marching Cubes Algorithm

The **Marching Cubes algorithm** is a widely-used technique for extracting a polygonal mesh of an **isosurface** from a 3-dimensional scalar field. An **isosurface** is a surface that represents points of equal value within a volume of space, allowing for the visualization of specific data values in 3D datasets.

This method is especially useful in fields such as:

- **Medical imaging**: Helps generate clear 3D representations of anatomical structures from CT or MRI scans.
- **Scientific simulations**: Aids in the visualization of phenomena like fluid dynamics.
- **Topographical map generation**: Visualizes terrain features based on elevation data.





### Steps for Implementing the Marching Cubes Algorithm

1. **Divide the volume into a grid of cubes**:  
   The first step is to divide the entire 3D scalar field (volume) into a grid of smaller cubes. Each cube will serve as the basic unit for evaluating where the isosurface intersects with the volume. The size of the cubes affects the resolution of the final mesh; smaller cubes lead to a more detailed representation of the surface.

2. **Determine the intersection of the isosurface with each cube**:  
   For each cube in the grid, identify how the isosurface intersects it. This is done by comparing the scalar values at the cube’s eight vertices to the target isosurface value. Based on these comparisons, determine which edges of the cube the isosurface crosses. This step establishes the boundary between regions where the scalar value is above or below the isosurface value.

3. **Use a predefined lookup table to triangulate the surface within each cube**:  
   After identifying the edges where the isosurface intersects the cube, a lookup table is used to determine how to form triangles that approximate the surface inside the cube. There are 256 possible ways the isosurface can intersect a cube, and the lookup table provides an efficient way to generate the correct set of triangles based on the intersection pattern.

4. **Combine the triangles from all cubes to form the final mesh**:  
   Once the triangles are generated for each cube, they are combined to create a continuous surface. The result is a polygonal mesh that represents the isosurface across the entire volume. This mesh can then be rendered or further processed, depending on the application.




### Understanding the Lookup Table in the Marching Cubes Algorithm

- **Each cube has 8 vertices, and each vertex can either be inside or outside the isosurface**:  
  In the Marching Cubes algorithm, each cube in the 3D grid consists of eight vertices. At each vertex, a scalar value is assigned. Depending on whether the scalar value at the vertex is above or below the target isosurface value, the vertex is considered to be either inside or outside the isosurface. This distinction is crucial for determining how the isosurface intersects the cube.

- **There are 2^8 = 256 possible combinations of how the vertices can intersect the isosurface**:  
  Since each of the 8 vertices can independently be either inside or outside the isosurface, there are 2^8 (256) possible ways these vertices can be configured. Each of these configurations corresponds to a different way the isosurface might intersect the cube, leading to different patterns of edge intersections.

- **Due to symmetry and rotational equivalences, the 256 configurations can be reduced to 15 unique cases**:  
  Although there are 256 possible configurations, many of them are similar due to symmetry or rotational equivalence. By taking these similarities into account, the number of unique intersection patterns can be reduced to just 15 cases. These 15 base cases are used to efficiently generate the appropriate triangles for any cube configuration using the lookup table.



<img src="./images/MarchingCubesVTKBook.png" width=800>

This figure appears originally in the VTK Textbook
https://vtk.org/vtk-textbook/


## Example

The file `bucky.raw`, included with the Marching Cubes example in the NVIDIA CUDA SDK, represents a 32x32x32 regular grid of 8-bit values.

When visualized using volume rendering in ParaView, the output looks like this:

<img src="./images/buckyVolumeRendering.png" width=800 alt="Volume Rendering of bucky.raw">

Next, after applying the Marching Cubes algorithm in ParaView with an isovalue of 127.5, the resulting surface is:

<img src="./images/buckySurface.png" width=800 alt="Surface Rendering using Marching Cubes">

> **Note**:  
> This image represents the surface where the scalar field has a value of 127.5.

Additionally, we can visualize the triangles generated by the Marching Cubes algorithm in wireframe mode:

<img src="./images/wireframe.png" width=800 alt="Wireframe of Marching Cubes Triangles">



## Parallelization: Where Does SYCL Come In?

- **Cube Processing**:  
  Each cube in the grid can be processed independently to determine where the isosurface intersects the cube. This independence makes it highly suitable for parallelization.

- **Vertex Interpolation**:  
  The interpolation calculations required to find the exact intersection points along the cube’s edges can be parallelized, speeding up the process of defining the isosurface.

- **Triangle Generation**:  
  Generating triangles based on the entries in the lookup table can also be performed in parallel, further improving efficiency by handling multiple cubes simultaneously.




## Organization of the Code

This code is based on the NVIDIA CUDA implementation of the Marching Cubes algorithm, available here:  
[https://github.com/NVIDIA/cuda-samples/tree/master/Samples/5_Domain_Specific/marchingCubes](https://github.com/NVIDIA/cuda-samples/tree/master/Samples/5_Domain_Specific/marchingCubes)

The following notes are from the original code from Nvidia in the repository linked above:

```
  Marching Cubes

  This sample extracts a geometric isosurface from a volume dataset using
  the marching cubes algorithm. It uses the scan (prefix sum) function from
  the Thrust library to perform stream compaction. Similar techniques can
  be used for other problems that require a variable-sized output per
  thread.

  For more information on Marching Cubes, see:
  http://local.wasp.uwa.edu.au/~pbourke/geometry/polygonise/
  http://en.wikipedia.org/wiki/Marching_cubes

  Volume data courtesy of:
  http://www9.informatik.uni-erlangen.de/External/vollib/

  For more information on the Thrust library:
  http://code.google.com/p/thrust/

  The algorithm consists of several stages:

  1. Execute the "classifyVoxel" kernel
     This evaluates the volume at the corners of each voxel and computes the
     number of vertices each voxel will generate.
     It is executed using one thread per voxel.
     It writes two arrays—voxelOccupied and voxelVertices—to global memory.
     voxelOccupied is a flag indicating if the voxel is non-empty.

  2. Scan the "voxelOccupied" array (using Thrust scan)
     Read back the total number of occupied voxels from GPU to CPU.
     This is the sum of the last value of the exclusive scan and the last
     input value.

  3. Execute the "compactVoxels" kernel
     This compacts the voxelOccupied array to remove empty voxels.
     This allows us to run the more complex "generateTriangles" kernel only
     on the occupied voxels.

  4. Scan the "voxelVertices" array
     This provides the start address for the vertex data of each voxel.
     We read back the total number of vertices generated from GPU to CPU.

     Note: By using a custom scan function, we could combine the above two
     scan operations into a single operation.

  5. Execute the "generateTriangles" kernel
     This runs only on the occupied voxels.
     It looks up the field values again and generates the triangle data,
     using the scan results to write the output to the correct addresses.
     The Marching Cubes lookup tables are stored in 1D textures.

  6. Render the geometry
     Using the number of vertices from the readback.
```

From these notes, we can identify three key kernels:  
- `classifyVoxel`  
- `compactVoxels`  
- `generateTriangles`

In the following sections, we will demonstrate how these kernels can be implemented using SYCL.



# SYCL Kernels

## Classify Voxel


### Description

The `classifyVoxel` kernel processes a 3D volume grid, classifying each voxel based on the field values stored at its corners. The kernel determines whether each voxel is intersected by an isosurface defined by a given isovalue. It outputs the number of triangle vertices generated for each voxel and whether the voxel is active (i.e., intersects the isosurface).

The output from this kernel (`voxelVerts` and `voxelOccupied`) will be used in later stages to efficiently generate the vertices and triangles that represent the isosurface.

### Key Operations

1. **Number of Vertices Determination**:
   - Using the `cubeindex`, the kernel looks up the number of vertices required to render the isosurface within this voxel (`numVerts`). This value is fetched from a precomputed `numVertsAcc` lookup table.

2. **Output Classification**:
   - The kernel writes the number of vertices (`voxelVerts[i]`) that will be generated for the voxel.
   - The kernel sets a flag (`voxelOccupied[i]`) indicating whether the voxel is intersected by the isosurface (i.e., whether `numVerts > 0`).


In [ ]:
%%writefile src/kernelClassifyVoxel.cpp

void classifyVoxel(uint *voxelVerts, 
                   uint *voxelOccupied, 
                   uchar *volume,
                   sycl::uint3 gridSize, 
                   sycl::uint3 gridSizeShift,
                   sycl::uint3 gridSizeMask, 
                   uint numVoxels,
                   sycl::float3 voxelSize, 
                   float isoValue,
                   sycl::accessor<uint, 1, sycl::access_mode::read> numVertsAcc,
                   sycl::accessor<uchar, 1, sycl::access_mode::read> volumeAcc,
                   sycl::id<3> idx) {

  uint i = idx[2] * gridSize.x() * gridSize.y() + idx[1] * gridSize.x() + idx[0];

  sycl::uint3 gridPos = calcGridPos(i, gridSizeShift, gridSizeMask);

  float field[8];
  field[0] = sampleVolume(volumeAcc, volume, gridPos, gridSize);
  field[1] = sampleVolume(volumeAcc, volume, gridPos + sycl::uint3(1, 0, 0), gridSize);
  field[2] = sampleVolume(volumeAcc, volume, gridPos + sycl::uint3(1, 1, 0), gridSize);
  field[3] = sampleVolume(volumeAcc, volume, gridPos + sycl::uint3(0, 1, 0), gridSize);
  field[4] = sampleVolume(volumeAcc, volume, gridPos + sycl::uint3(0, 0, 1), gridSize);
  field[5] = sampleVolume(volumeAcc, volume, gridPos + sycl::uint3(1, 0, 1), gridSize);
  field[6] = sampleVolume(volumeAcc, volume, gridPos + sycl::uint3(1, 1, 1), gridSize);
  field[7] = sampleVolume(volumeAcc, volume, gridPos + sycl::uint3(0, 1, 1), gridSize);

  uint cubeindex;
  cubeindex = uint(field[0] < isoValue);
  cubeindex += uint(field[1] < isoValue) * 2;
  cubeindex += uint(field[2] < isoValue) * 4;
  cubeindex += uint(field[3] < isoValue) * 8;
  cubeindex += uint(field[4] < isoValue) * 16;
  cubeindex += uint(field[5] < isoValue) * 32;
  cubeindex += uint(field[6] < isoValue) * 64;
  cubeindex += uint(field[7] < isoValue) * 128;

  uint numVerts = numVertsAcc[cubeindex];

  if (i < numVoxels) {
    voxelVerts[i] = numVerts;
    voxelOccupied[i] = (numVerts > 0);
  }
}

### Launch classifyVoxel

In [ ]:
%%writefile src/kernelLaunchClassifyVoxel.cpp

extern "C" 
void launch_classifyVoxel(sycl::queue &q, 
                          sycl::range<3> globalRange,
                          uint *voxelVerts, 
                          uint *voxelOccupied, 
                          uchar *volume,
                          sycl::uint3 gridSize, 
                          sycl::uint3 gridSizeShift,
                          sycl::uint3 gridSizeMask, 
                          uint numVoxels,
                          sycl::float3 voxelSize, 
                          float isoValue) 
{
  q.submit([&](sycl::handler &h) 
  {
    auto numVertsAcc = numVertsTableBuf -> get_access<sycl::access_mode::read>(h);
    auto volumeAcc = volumeBuf -> get_access<sycl::access_mode::read>(h);
        
    h.parallel_for(globalRange, [=](sycl::id<3> idx) 
    {
      classifyVoxel(voxelVerts, voxelOccupied, volume, gridSize,
                    gridSizeShift, gridSizeMask, numVoxels, voxelSize,
                    isoValue, numVertsAcc, volumeAcc, idx);        
    }); 
  }).wait();
}

## Compact voxels


### Description: 

The `compactVoxels` kernel compacts a list of active (or occupied) voxels into a contiguous array. This is typically used  to minimize processing and memory usage by only considering voxels that contain part of the isosurface. Each voxel is represented by a unique index within a 3D grid, and the kernel utilizes a prefix-sum (scan) result to determine the new, compacted position of each active voxel.

The compacted array (`compactedVoxelArray`) produced by this kernel is used in the next stage of the **Marching Cubes** algorithm to limit the number of voxels processed during triangle generation. This optimization reduces the computational load by excluding empty voxels from further calculations.

### Key Operations

1. **Occupancy Check**:
   - For voxels marked as occupied (`voxelOccupied[i]`), the kernel proceeds to store the voxel's index `i` into a new position in `compactedVoxelArray`. The position in `compactedVoxelArray` is determined using the `voxelOccupiedScan` array, which contains the prefix-sum results (scan) of the `voxelOccupied` array.

2. **Compaction**:
   - The compaction step only occurs for active voxels, ensuring that `compactedVoxelArray` contains a tightly packed list of only the indices of occupied voxels.
     

In [ ]:
%%writefile src/kernelCompactVoxels.cpp

void compactVoxels(uint *compactedVoxelArray, 
                   uint *voxelOccupied,
                   uint *voxelOccupiedScan, 
                   uint numVoxels, 
                   sycl::id<3> idx) 
{
  uint i = idx[2] * numVoxels * numVoxels + idx[1] * numVoxels + idx[0];
  if (i >= numVoxels) return;
  
  if (voxelOccupied[i]) {
    compactedVoxelArray[voxelOccupiedScan[i]] = i;
  }
}

### Launch compact voxels

In [ ]:
%%writefile src/kernelLaunchCompactVoxels.cpp

extern "C"
void launch_compactVoxels(sycl::queue &q, 
                          sycl::range<3> globalRange,
                          uint *compactedVoxelArray, 
                          uint *voxelOccupied,
                          uint *voxelOccupiedScan, 
                          uint numVoxels) 
{
  q.submit([&](sycl::handler &h) 
  {      
    h.parallel_for(globalRange, [=](sycl::id<3> idx) 
    {
      compactVoxels(compactedVoxelArray, voxelOccupied, voxelOccupiedScan, numVoxels, idx);
    }); 
  }).wait(); // Wait for the kernel to complete
}

## generateTriangles

## Kernel Description:

The `generateTriangles` kernel is used to generate triangle vertices based on active voxels identified in a 3D grid. It processes each active voxel to compute the intersections of the isosurface with the voxel edges, generating triangle vertices for each intersected voxel and storing them in an output buffer (`pos`). The final result is a collection of triangle vertices that represent the surface extracted from the 3D scalar field.

The kernel generates triangles for all active voxels based on the computed isosurface intersections. These triangles are then used for rendering or further processing in applications such as 3D visualization or mesh generation.

### Key Operations

1. **Vertex Interpolation**:
   - Interpolated vertices (`vertlist`) and normals (`normlist`) are computed for each edge intersected by the isosurface using `vertexInterp2`. This step calculates the exact location of the intersection points along the edges of the voxel.

2. **Triangle Generation**:
   - The `triTableAcc` is used to determine the order in which vertices should be connected to form triangles. For each triangle, vertices are generated and stored in the `pos` buffer as `sycl::float4` values, where the fourth component is set to `1.0`.

3. **Output to Global Memory**:
   - The generated triangle vertices are stored in the output array `pos` using the index derived from `numVertsScanned[voxel]`. This ensures that the vertices are correctly placed according to the scan results and preserves the compacted structure of the active voxel array.



In [ ]:
%%writefile src/kernelGenerateTriangles.cpp

void generateTriangles(sycl::float4 *pos, 
        sycl::float4 *norm, 
        uint *compactedVoxelArray, 
        uint *numVertsScanned,
        sycl::uint3 gridSize, 
        sycl::uint3 gridSizeShift, 
        sycl::uint3 gridSizeMask, 
        sycl::float3 voxelSize,
        float isoValue, 
        uint activeVoxels, 
        uint maxVerts,
        sycl::accessor<uint, 1, sycl::access_mode::read> triTableAcc,
        sycl::accessor<uint, 1, sycl::access_mode::read> numVertsAcc,
        sycl::id<3> idx)  
{
  uint i = idx[2] * gridSize.x() * gridSize.y() + idx[1] * gridSize.x() + idx[0];

  if (i >= activeVoxels ) {
    return;  // Exit early for threads that are out of range
  }
         
  uint voxel = compactedVoxelArray[i];

  // Compute position in 3D grid
  sycl::uint3 gridPos = calcGridPos(voxel, gridSizeShift, gridSizeMask);

  sycl::float3 p;
  p.x() = -1.0f + (gridPos.x() * voxelSize.x());
  p.y() = -1.0f + (gridPos.y() * voxelSize.y());
  p.z() = -1.0f + (gridPos.z() * voxelSize.z());

  // Calculate cell vertex positions
  sycl::float3 v[8];
  v[0] = p;
  v[1] = p + sycl::float3(voxelSize.x(), 0, 0);
  v[2] = p + sycl::float3(voxelSize.x(), voxelSize.y(), 0);
  v[3] = p + sycl::float3(0, voxelSize.y(), 0);
  v[4] = p + sycl::float3(0, 0, voxelSize.z());
  v[5] = p + sycl::float3(voxelSize.x(), 0, voxelSize.z());
  v[6] = p + sycl::float3(voxelSize.x(), voxelSize.y(), voxelSize.z());
  v[7] = p + sycl::float3(0, voxelSize.y(), voxelSize.z());

  // Evaluate field values
  sycl::float4 field[8];
  field[0] = fieldFunc4(v[0]);
  field[1] = fieldFunc4(v[1]);
  field[2] = fieldFunc4(v[2]);
  field[3] = fieldFunc4(v[3]);
  field[4] = fieldFunc4(v[4]);
  field[5] = fieldFunc4(v[5]);
  field[6] = fieldFunc4(v[6]);
  field[7] = fieldFunc4(v[7]);

  // Recalculate cube index
  uint cubeindex;
  cubeindex = uint(field[0].w() < isoValue);
  cubeindex += uint(field[1].w() < isoValue) * 2;
  cubeindex += uint(field[2].w() < isoValue) * 4;
  cubeindex += uint(field[3].w() < isoValue) * 8;
  cubeindex += uint(field[4].w() < isoValue) * 16;
  cubeindex += uint(field[5].w() < isoValue) * 32;
  cubeindex += uint(field[6].w() < isoValue) * 64;
  cubeindex += uint(field[7].w() < isoValue) * 128;

  // find the vertices where the surface intersects the cube
  sycl::float3 vertlist[12];
  sycl::float3 normlist[12];

  vertexInterp2(isoValue, v[0], v[1], field[0], field[1], vertlist[0], normlist[0]);
  vertexInterp2(isoValue, v[1], v[2], field[1], field[2], vertlist[1], normlist[1]);
  vertexInterp2(isoValue, v[2], v[3], field[2], field[3], vertlist[2], normlist[2]);
  vertexInterp2(isoValue, v[3], v[0], field[3], field[0], vertlist[3], normlist[3]);
  vertexInterp2(isoValue, v[4], v[5], field[4], field[5], vertlist[4], normlist[4]);
  vertexInterp2(isoValue, v[5], v[6], field[5], field[6], vertlist[5], normlist[5]);
  vertexInterp2(isoValue, v[6], v[7], field[6], field[7], vertlist[6], normlist[6]);
  vertexInterp2(isoValue, v[7], v[4], field[7], field[4], vertlist[7], normlist[7]);
  vertexInterp2(isoValue, v[0], v[4], field[0], field[4], vertlist[8], normlist[8]);
  vertexInterp2(isoValue, v[1], v[5], field[1], field[5], vertlist[9], normlist[9]);
  vertexInterp2(isoValue, v[2], v[6], field[2], field[6], vertlist[10], normlist[10]);
  vertexInterp2(isoValue, v[3], v[7], field[3], field[7], vertlist[11], normlist[11]);

  // Output triangle vertices
  uint numVerts = numVertsAcc[cubeindex];

  for (int j = 0; j < numVerts; j++) 
  {
    uint edge = triTableAcc[cubeindex * 16 + j];
    uint index = numVertsScanned[voxel] + j;

    if (index < maxVerts) 
    {              
      pos[index] = sycl::float4(vertlist[edge], 1.0f);
      norm[index] = sycl::float4(normlist[edge], 0.0f);
    }
  }   
}

### Launch generateTriangles kernel

In [ ]:
%%writefile src/kernelLaunchGenerateTriangles.cpp

extern "C" 
void launch_generateTriangles(sycl::queue &q, sycl::range<3> globalRange,
                              sycl::float4 *pos, 
                              sycl::float4 *norm,
                              uint *compactedVoxelArray, 
                              uint *numVertsScanned,
                              sycl::uint3 gridSize, 
                              sycl::uint3 gridSizeShift,
                              sycl::uint3 gridSizeMask, 
                              sycl::float3 voxelSize,
                              float isoValue, 
                              uint activeVoxels, 
                              uint maxVerts) 
{
  q.submit([&](sycl::handler &h) 
  {  
    auto triTableAcc = triTableBuf->get_access<sycl::access_mode::read>(h);
    auto numVertsAcc = numVertsTableBuf->get_access<sycl::access_mode::read>(h);

    h.parallel_for(globalRange, [=](sycl::id<3> idx) 
    {
      generateTriangles(pos, norm, compactedVoxelArray, numVertsScanned,
                            gridSize, gridSizeShift, gridSizeMask, voxelSize,
                            isoValue, activeVoxels, maxVerts, triTableAcc,
                            numVertsAcc, idx);
    });
  }).wait();
}

#### Build and Run
Select the cell below and click __Run__ ▶ to compile the code:

In [ ]:
%%writefile ./run-sycl.sh
#!/bin/bash 
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
[ ! -d build ] && mkdir build
cd build
echo "*** Starting build ***"
icpx -fsycl ../src/marchingCubes.cpp ../src/marchingCubesKernels.cpp -o marchingCubes
echo "*** Build finished ***"
echo "*** Running application ***"
if [ $? -eq 0 ]; then ./marchingCubes; fi
echo "*** Application finished running ***"


Select the cell below and click __Run__ ▶ to execute the code on selected device:

In [ ]:
!chmod u+x ./run-sycl.sh &&./run-sycl.sh